In [1]:
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import sqrt

In [2]:
def extract_data(path,create_test=None):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]

    file=os.listdir(path)

    for entity in file:
        try:
            entityPath=os.listdir(f"{path}/{entity}")
            for i,data in enumerate(entityPath):
                try:
                    current_img=cv2.imread(f"{path}/{entity}/{data}")
                    if current_img is not None:
                        if (create_test is not None and ((len(entityPath)>1) and (i==0))):
                            X_test.append(current_img)
                            y_test.append(entity)
                        else:
                            X_train.append(current_img)
                            y_train.append(entity)
                    else:
                        print(f"{path}/{entity}/{data} is None type")
                except Exception as e:
                    print(f"Erreur avec {path}/{entity}/{data} : {e}")
        except Exception as ex:
            print(f"Erreur avec {path}/{entity} : {ex}")
    
    if create_test is not None:
        return X_train,y_train,X_test,y_test
    else:
        return X_train,y_train

In [3]:
def imgProcess(img,withTorch=None):
    if not isinstance(img,np.ndarray):
        print("convertion de l'img")
        img=cv2.imread(img)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #RGB -> Gray -> dim(255,255,3)=255*255*3 val de couleur = 195075 vals -> dim(255,255,1) -> 255*255*1 val de couleur = 65025 vals pour l'exemple du 255*255
    img=cv2.resize(img,(100,100)) #dim -> dim(100,100,1) -> 100*100*1 = 10000 val de couleur
    if withTorch is None:
        img=img.flatten() #2D -> 1D vec de (10000,1)
    else:
        img=img.astype(np.float32)/255.0 #rescale entre 0 et 1
        img=img[np.newaxis,:,:] #ajout d'une dimension pour la couleur
        img=torch.tensor(img).unsqueeze(0) #ajout de la dimension de batch necessaire pour pytorch
    return img

In [4]:
def distEuclid(img1,img2): #p1(a1,a2,a3...,a10000) p2(b1,b2,b3...,b10000) car chaque image a 10000 val de couleur
    #img1 = img1.astype(np.int32) #pour avoir des valeur négatives sinon en uint8 c'est de 0 à 255 (pas negatif)
    #img2 = img2.astype(np.int32)
    diff=img1.astype(np.int32)-img2.astype(np.int32) #opti

    #distance euclidienne entre deux points (en 2D) p1(x1,y1) et p2(x2,y2): d=hypothénuse du triangle rectangle
    #ainsi d²=a²+b² et d=sqrt(a²+b²) avec a=abs(x1-x2) et b=abs(y1-y2) DONC -> d=sqrt((x1-x2)²+(y1-y2)²)
    #en n dim : d=sqrt((a1-b1)²+(a2-b2)²...+(an-bn)²) = sqrt(sum[1;N]((a-b)²))
    #return sqrt(sum((a-b)**2 for a,b in zip(img1,img2))) #zip -> pour deux listes
    return np.sqrt(np.sum(diff**2)) #opti

In [5]:
def knn_predict(X_train,y_train, newImg,k):
    distances=[]
    for i in range(len(X_train)):
        dist=distEuclid(X_train[i],newImg) #on calcule la distance entre la nouvelle image et toutes les autres images
        distances.append((dist,y_train[i])) #on stock la distance avec le label de chaque image à laquelle on a calculé la distance
    
    distances.sort(key=lambda x: x[0]) #on trie la liste de distances dans l'ordre croissant comme ça les derniers termes sont les plus proche de la nouvelle image
    k_voisins=distances[:k] #on prend les k plus proches voisins

    counts={} #dictionnaire pour compter les labels les plus proches et leur occurences
    for _,label in k_voisins: #dans le tableau c'est [distance,label], ici on veut que le label donc _
        counts[label]=counts.get(label,0)+1 #.get(a,b) = prend la val a si existe sinon prend la val b

    return max(counts,key=counts.get) #compare les valeurs mais renvoie le label du dictionnaire

In [6]:
def knn_evaluate(processed_train,y_train,processed_test,y_test,k,n):
    knn_results=[]
    true=0
    if n is None:
        n=len(processed_test)

    print(f"Number of sample set to : {n}")
    
    for i,img in enumerate(processed_test[:n]):
        print(f"Image KNN : {i}")
        knn_results.append(knn_predict(processed_train,y_train,processed_test[i],k))
        if knn_results[i]==y_test[i]:
            true+=1
    print(f"Accuracy (%) : {100*true/n}")

    return knn_results

In [7]:
def KNN(X_train,y_train,X_test,y_test,k=3,n=None):
    processed_train=[]
    for img in X_train:
        processed_train.append(imgProcess(img))

    processed_test=[]
    for img in X_test:
        processed_test.append(imgProcess(img))
    print("Processing done.\n")

    knn_result=knn_evaluate(processed_train,y_train,processed_test,y_test,k,n)
    return knn_result

In [8]:
def imgPadding(img,padding):
    size=img.shape[0]
    if padding is not None:
        size=size+2*padding
        newImg=np.zeros((size,size,3),dtype=np.uint8)
        for x in range(padding,size-padding):
            for y in range(padding,size-padding):
                newImg[x][y]=img[x-padding][y-padding]
    else:
        newImg=img
    return newImg

In [9]:
def max_pooling(img,dim=2,padding=None,stride=1):
    if padding is not None:
        img=imgPadding(img,padding)
    
    newDim=((img.shape[0]-dim)//stride)+1
    newImg=np.zeros([newDim,newDim,3])
    
    for rgb in range(img.shape[2]):
        for x in range(0,(img.shape[0]//dim)*dim,stride):
            for y in range(0,(img.shape[0]//dim)*dim,stride):
                max=0
                for h in range(dim):
                    for l in range(dim):
                        if img[x+l][y+h][rgb]>max:
                            max=img[x+l][y+h][rgb]
                newImg[((x-dim)//stride)+1][((y-dim)//stride)+1][rgb]=max
    
    return newImg

In [10]:
def average_pooling(img,dim=2,padding=None,stride=1):
    if padding is not None:
        img=imgPadding(img,padding)
    
    newDim=((img.shape[0]-dim)//stride)+1
    newImg=np.zeros([newDim,newDim,3])
    
    for rgb in range(img.shape[2]):
        for x in range(0,(img.shape[0]//dim)*dim,stride):
            for y in range(0,(img.shape[0]//dim)*dim,stride):
                somme=0
                for h in range(dim):
                    for l in range(dim):
                        somme+=img[x+l][y+h][rgb]
                newImg[((x-dim)//stride)+1][((y-dim)//stride)+1][rgb]=somme//dim*dim
    
    return newImg

In [14]:
def make_model(nb_classes):
    model=nn.Sequential(
        nn.Conv2d(1,16,kernel_size=3,padding=1), #(1,100,100) -> (16,100,100)
        nn.ReLU(),
        nn.MaxPool2d(2,2), #(16,100,100) -> ((100-2)//2)+1 = 49+1 = 50 donc -> (16,50,50)
        nn.Conv2d(16,32,kernel_size=3,padding=1), #(16,50,50) -> (32,50,50)
        nn.ReLU(),
        nn.MaxPool2d(2,2), #(32,50,50) -> ((50-2)//2)+1 = 25 -> (32,25,25)
        nn.Flatten(), #(32,25,25) -> (32*25*25) = (20000)
        #nn.Dropout(0.3) #pour éviter l'overfitting
        nn.Linear(20000,128),
        nn.ReLU(),
        nn.Linear(128,nb_classes)
    )
    return model

In [15]:
def CNN(X_train,y_train,X_test,y_test,model=None):
    if model is None:
        model=make_model(len(list(set(y_train)))) #list(set(y_train)) sort les val uniques

In [16]:
print(make_model(2))

Sequential(
  (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=20000, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=2, bias=True)
)


In [24]:
for i,img in enumerate(knn_results):
    print(f"{i} : Predict: {img}, reel: {y_test[i]}")

0 : Predict: Augustin, reel: Augustin
1 : Predict: Augustin, reel: Augustin
2 : Predict: Augustin, reel: Augustin
3 : Predict: Augustin, reel: Augustin
4 : Predict: Augustin, reel: Macha
